In [1]:
import os
import pandas as pd
import tarfile
# import pqdm

In [2]:
# df = pd.read_csv('filtered_trivy_output.csv')
repo_files_dir = "HelmCharts"
remediated_repo_files_dir = "RemediatedCharts"

In [6]:
with open('success.txt','r') as file:
    successful_repos = [line.rstrip() for line in file]

In [ ]:
# Unzip the tgz files

In [34]:
def extract_and_rename(tgz_path):
    
    tgz_name = os.path.basename(tgz_path)
    sub_directory_name = '-'.join(tgz_name.split('-')[:-1])
    full_tgz_path = repo_files_dir+"/"+ tgz_path
    
    with tarfile.open(full_tgz_path, "r:gz") as tar:
        tar.extractall(path=remediated_repo_files_dir)
    
    extracted_directory_path = remediated_repo_files_dir + '/' + sub_directory_name    
    values_yaml_path = os.path.join(extracted_directory_path, "values.yaml")

    return values_yaml_path

In [37]:
# List to store the directories that need to be deleted at the end
directories_to_delete = []
final_yaml_file_locations = []
# Add progress bar with tqdm
total_files = len(successful_repos)

for row in successful_repos:
    tgz_path = row
    tgz_path_extracted = tgz_path
    extraction_dir = extract_and_rename(tgz_path)
    if extraction_dir:
        final_yaml_file_locations.append(extraction_dir)
        original_path = repo_files_dir+'/'+tgz_path
        directories_to_delete.append(original_path)

In [ ]:
# After processing all files, delete all extracted directories
# logging.info("Deleting all extracted directories...")
# for directory in directories_to_delete:
#     if os.path.exists(directory):
#         delete_directory_manually(directory)
#     else:
#         logging.warning(f"Directory {directory} does not exist or has already been deleted.")

In [40]:
# List to store the directories that need to be deleted at the end
# directories_to_delete = []

# # Add progress bar with tqdm
# total_files = len(df_unique)
# with tqdm(total=total_files, desc="Processing files", unit="file") as pbar:
#     # Iterate over each unique row in the DataFrame
#     for index, row in df_unique.iterrows():
#         # Split the Target column to get the tgz path
#         target_value = row['Target']
#         tgz_path = target_value.split(":")[0]  # Get the tgz part before the colon

#         # Process the tgz file and collect the directory to delete later
#         extraction_dir = extract_and_rename(tgz_path)
#         if extraction_dir:
#             directories_to_delete.append(extraction_dir)

#         # Update progress bar after each file
#         pbar.update(1)

# # After processing all files, delete all extracted directories
# logging.info("Deleting all extracted directories...")
# for directory in directories_to_delete:
#     if os.path.exists(directory):
#         delete_directory_manually(directory)
#     else:
#         logging.warning(f"Directory {directory} does not exist or has already been deleted.")